# Multi-Echo Denoising with `tedana`

In this analysis tutorial, we will use `tedana` {cite:p}`DuPre2021` to perform multi-echo denoising.

Specifically, we will use {py:func}`tedana.workflows.tedana_workflow`.

In [1]:
import json
import os
from glob import glob
from pprint import pprint

import pandas as pd
from IPython.display import HTML, display
from repo2data.repo2data import Repo2Data
from tedana import workflows

# Install the data if running locally, or point to cached data if running on neurolibre
DATA_REQ_FILE = os.path.join("../binder/data_requirement.json")

# Download data
repo2data = Repo2Data(DATA_REQ_FILE)
data_path = repo2data.install()
data_path = os.path.abspath(data_path[0])

---- repo2data starting ----
/opt/hostedtoolcache/Python/3.10.17/x64/lib/python3.10/site-packages/repo2data
Config from file :
../binder/data_requirement.json
Destination:
./../data/multi-echo-data-analysis

Info : ./../data/multi-echo-data-analysis already downloaded


/opt/hostedtoolcache/Python/3.10.17/x64/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
func_dir = os.path.join(data_path, "func/")
data_files = [
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-1_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-2_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-3_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-4_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
]
echo_times = [12.0, 28.0, 44.0, 60.0]
mask_file = os.path.join(
    func_dir, "sub-04570_task-rest_space-scanner_desc-brain_mask.nii.gz"
)
confounds_file = os.path.join(
    func_dir, "sub-04570_task-rest_desc-confounds_timeseries.tsv"
)

out_dir = os.path.join(data_path, "tedana")

In [3]:
workflows.tedana_workflow(
    data_files,
    echo_times,
    out_dir=out_dir,
    mask=mask_file,
    prefix="sub-04570_task-rest_space-scanner",
    fittype="curvefit",
    tedpca="mdl",
    verbose=True,
    gscontrol=["mir"],
)

INFO     tedana:tedana_workflow:596 Using output directory: /home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/tedana


INFO     tedana:tedana_workflow:615 Initializing and validating component selection tree


WARNING  component_selector:validate_tree:146 Decision tree includes fields that are not used or logged ['_comment']


INFO     component_selector:__init__:333 Performing component selection with tedana_orig_decision_tree


INFO     component_selector:__init__:334 Very similar to the decision tree designed by Prantik Kundu


...

INFO     html_report:_update_template_bokeh:198 S0 files exist: True


INFO     html_report:_update_template_bokeh:199 RMSE files exist: True


INFO     tedana:tedana_workflow:1142 Workflow completed


INFO     utils:log_newsletter_info:705 Don't forget to subscribe to the tedana newsletter for updates! This is a very low volume email list.


INFO     utils:log_newsletter_info:709 https://groups.google.com/g/tedana-newsletter


The tedana workflow writes out a number of files.

In [4]:
out_files = sorted(glob(os.path.join(out_dir, "*")))
out_files = [os.path.basename(f) for f in out_files]
print("\n".join(out_files))

figures
sub-04570_task-rest_space-scanner_S0map.nii.gz
sub-04570_task-rest_space-scanner_T2starmap.nii.gz
sub-04570_task-rest_space-scanner_dataset_description.json
sub-04570_task-rest_space-scanner_desc-ICAAcceptedMIRDenoised_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAccepted_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAccepted_stat-z_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAveragingWeights_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICACrossComponent_metrics.json
sub-04570_task-rest_space-scanner_desc-ICAMIRDenoised_mixing.tsv
sub-04570_task-rest_space-scanner_desc-ICAS0_stat-F_statmap.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAT2_stat-F_statmap.nii.gz
sub-04570_task-rest_space-scanner_desc-ICA_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICA_decision_tree.json
sub-04570_task-rest_space-scanner_desc-ICA_decomposition.json
sub-04570_task-rest_space-scanner_desc-ICA_mixing.tsv
sub-04570_task-rest_space

In [5]:
metrics = pd.read_table(
    os.path.join(out_dir, "sub-04570_task-rest_space-scanner_desc-tedana_metrics.tsv")
)

In [6]:
def color_rejected_red(series):
    """Color rejected components red."""
    return [
        "color: red" if series["classification"] == "rejected" else "" for v in series
    ]


metrics.style.apply(color_rejected_red, axis=1)

,Component,kappa,rho,variance explained,normalized variance explained,countsigFT2,countsigFS0,dice_FT2,dice_FS0,countnoise,signal-noise_t,signal-noise_p,d_table_score,optimal sign,varex kappa ratio,d_table_score_node20,Var Exp of rejected to accepted,classification,classification_tags
0,ICA_00,22.146228,24.116454,0.775255,0.012156,1216,941,0.158952,0.224756,1168,-6.424964,0.000000,29.200000,-1,0.587732,nan,nan,rejected,Unlikely BOLD
1,ICA_01,51.230575,24.753611,23.506897,0.248546,5353,2484,0.513640,0.191117,965,0.000000,0.000000,5.800000,-1,7.703721,4.000000,16.016520,accepted,Likely BOLD
2,ICA_02,22.893331,14.901836,0.357921,0.005175,1145,422,0.292726,0.000000,1144,6.592229,0.000000,19.400000,-1,0.262490,nan,10.316781,accepted,Low variance
3,ICA_03,44.193614,19.191723,1.333745,0.017837,4209,1879,0.483883,0.270373,1179,0.000000,0.000000,10.200000,-1,0.506696,7.000000,2.511483,accepted,Likely BOLD
4,ICA_04,27.567406,20.737956,0.821346,0.011342,1796,1399,0.346452,0.308638,1165,1.168751,0.251138,14.700000,-1,0.500225,11.600000,8.002623,accepted,Likely BOLD
5,ICA_05,44.379684,15.646069,0.934832,0.012529,2517,592,0.454771,0.103478,1056,6.697817,0.000000,4.800000,1,0.353658,4.200000,9.192120,accepted,Likely BOLD
6,ICA_06,30.018327,23.605399,1.568946,0.018061,2068,1258,0.329256,0.241267,1218,0.000000,0.000000,18.000000,1,0.877518,13.000000,40.951937,accepted,Accept borderline
7,ICA_07,22.990509,14.921279,0.419216,0.006445,967,365,0.238532,0.000000,1223,0.000000,0.000000,28.900000,-1,0.306142,nan,6.227725,accepted,Low variance
8,ICA_08,21.753897,14.043088,0.276220,0.004194,1029,276,0.315582,0.000000,1099,8.598623,0.000000,17.800000,-1,0.213183,nan,3.232917,accepted,Low variance
9,ICA_09,31.741634,18.654378,0.739212,0.008564,1903,723,0.328529,0.187891,1225,3.541223,0.001735,16.800000,1,0.390998,12.200000,24.554134,accepted,Likely BOLD


In [7]:
with open(
    os.path.join(out_dir, "sub-04570_task-rest_space-scanner_desc-tedana_metrics.json"),
    "r",
) as fo:
    data = json.load(fo)

first_five_keys = list(data.keys())[:5]
reduced_data = {k: data[k] for k in first_five_keys}
pprint(reduced_data)

{'Component': {'Description': 'The unique identifier of each component. This '
                              'identifier matches column names in the mixing '
                              'matrix TSV file.',
               'LongName': 'Component identifier'},
 'classification_tags': {'Description': 'A single tag or a comma separated '
                                        'list of tags to describe why a '
                                        'component received its classification',
                         'LongName': 'Component classification tags'},
 'countnoise': {'Description': "Number of 'noise' voxels (voxels highly "
                               'weighted for component, but not from clusters) '
                               'from each component.',
                'LongName': 'Noise voxel count',
                'Units': 'voxel'},
 'countsigFS0': {'Description': 'Number of significant voxels from the '
                                'cluster-extent thresholded S0 model '

In [8]:
df = pd.DataFrame.from_dict(data, orient="index")
df = df.fillna("n/a")
display(HTML(df.to_html()))

,Description,LongName,Units,Levels
Component,The unique identifier of each component. This identifier matches column names in the mixing matrix TSV file.,Component identifier,n/a,n/a
classification_tags,A single tag or a comma separated list of tags to describe why a component received its classification,Component classification tags,n/a,n/a
countnoise,"Number of 'noise' voxels (voxels highly weighted for component, but not from clusters) from each component.",Noise voxel count,voxel,n/a
countsigFS0,Number of significant voxels from the cluster-extent thresholded S0 model F-statistic map for each component.,S0 model F-statistic map significant voxel count,voxel,n/a
countsigFT2,Number of significant voxels from the cluster-extent thresholded T2 model F-statistic map for each component.,T2 model F-statistic map significant voxel count,voxel,n/a
d_table_score,"Summary score compiled from five metrics, with smaller values (i.e., higher ranks) indicating more BOLD dependence and less noise.",Decision table score,arbitrary,n/a
dice_FS0,Dice value of cluster-extent thresholded maps of S0-model betas and F-statistics.,S0 model beta map-F-statistic map Dice similarity index,arbitrary,n/a
dice_FT2,Dice value of cluster-extent thresholded maps of T2-model betas and F-statistics.,T2 model beta map-F-statistic map Dice similarity index,arbitrary,n/a
kappa,"A pseudo-F-statistic indicating TE-dependence of the component. This metric is calculated by computing fit to the TE-dependence model at each voxel, and then performing a weighted average based on the voxel-wise weights of the component.",Kappa,arbitrary,n/a
normalized variance explained,Normalized variance explained in the optimally combined data of each component.On a scale from 0 to 1.,Normalized variance explained,arbitrary,n/a


In [9]:
report = os.path.join(out_dir, "tedana_report.html")
with open(report, "r") as fo:
    report_data = fo.read()

figures_dir = os.path.relpath(os.path.join(out_dir, "figures"), os.getcwd())
report_data = report_data.replace("./figures", figures_dir)

display(HTML(report_data))

FileNotFoundError: [Errno 2] No such file or directory: '/home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/tedana/tedana_report.html'